In [14]:
import pandas as pd


def read_data():
    data = pd.read_csv('data.csv')
    columns = ['time', 'PV(kWh)', 'WP(kWh)', 'load(kWh)']
    data.columns = columns
    return data

def compiling_data():
    data = read_data()
    data_copy = data.copy()

    data_copy['time'] = pd.to_datetime(data_copy['time'])

    data_copy['Total PVWP'] = data_copy['PV(kWh)'] + data_copy['WP(kWh)']
    data_copy['Total Buy(kWh)'] = data_copy.apply(lambda row: row['load(kWh)'] if row['Total PVWP'] > row['load(kWh)'] else row['Total PVWP'], axis=1)
    data_copy['Total Buy_PV'] = data_copy.apply(lambda row: row['Total Buy(kWh)']*(row['PV(kWh)']/row['Total PVWP']) if row['Total PVWP'] > row['load(kWh)'] else row['PV(kWh)'], axis=1)
    data_copy['Total Buy_WP'] = data_copy.apply(lambda row: row['Total Buy(kWh)']*(row['WP(kWh)']/row['Total PVWP']) if row['Total PVWP'] > row['load(kWh)'] else row['WP(kWh)'], axis=1)
    data_copy['Excess Electricity'] = data_copy.apply(lambda row: row['Total PVWP'] - row['load(kWh)'] if row['Total PVWP'] > row['load(kWh)'] else 0, axis=1)

    data_copy['Excess Electricity_PV'] = data_copy.apply(lambda row: 0 if row['Excess Electricity'] == 0 else row['PV(kWh)'] - row['Total Buy_PV'], axis=1)
    data_copy['Excess Electricity_WP'] = data_copy.apply(lambda row: 0 if row['Excess Electricity'] == 0 else row['WP(kWh)'] - row['Total Buy_WP'], axis=1)

    return data_copy

def data_merge():
    data_copy = compiling_data()

    total_load = data_copy['load(kWh)'].sum()

    df_merge = pd.DataFrame()
    df_merge['Total Load (kWh)'] = [total_load]
    df_merge['Total Purchased Electricity'] = [data_copy['Total Buy(kWh)'].sum()]
    df_merge['Merge_PVWP'] = [data_copy['Total PVWP'].sum()]
    df_merge['Total_RE%'] = df_merge['Total Purchased Electricity'] / df_merge['Total Load (kWh)']*100 if total_load != 0 else 0
    df_merge['Total Purchased Electricity_PVWP'] = [data_copy['Total Buy_PV'].sum() + data_copy['Total Buy_WP'].sum()]
    df_merge['Net Renewable_RE%'] = df_merge['Total Purchased Electricity_PVWP'] / df_merge['Total Load (kWh)']*100 if total_load != 0 else 0
    df_merge['Total Purchased Electricity_PV'] = [data_copy['Total Buy_PV'].sum()]
    df_merge['Total Purchased Electricity_WP'] = [data_copy['Total Buy_WP'].sum()]
    df_merge['Total Excess Electricity'] = [data_copy['Excess Electricity'].sum()]
    df_merge['Total Excess Electricity_PV'] = [data_copy['Excess Electricity_PV'].sum()]
    df_merge['Total Excess Electricity_WP'] = [data_copy['Excess Electricity_WP'].sum()]

    df_merge = df_merge.round(2)
    return df_merge


df_merge = data_merge()
df_merge

,Total Load (kWh),Total Purchased Electricity,Merge_PVWP,Total_RE%,Total Purchased Electricity_PVWP,Net Renewable_RE%,Total Purchased Electricity_PV,Total Purchased Electricity_WP,Total Excess Electricity,Total Excess Electricity_PV,Total Excess Electricity_WP
0,41108769.0,21308518.75,23620975.63,51.83,21308518.75,51.83,9066889.44,12241629.32,2312456.87,1016954.43,1295502.44
